# Day 13: Transparent Origami

[https://adventofcode.com/2021/day/13](https://adventofcode.com/2021/day/13)

## Description

### Part One

You reach another volcanically active part of the cave. It would be nice if you could do some kind of thermal imaging so you could tell ahead of time which caves are too hot to safely enter.

Fortunately, the submarine seems to be equipped with a thermal camera! When you activate it, you are greeted with:

    Congratulations on your purchase! To activate this infrared thermal imaging
    camera system, please enter the code found on page 1 of the manual.
    

Apparently, the Elves have never used this feature. To your surprise, you manage to find the manual; as you go to open it, page 1 falls out. It's a large sheet of [transparent paper](https://en.wikipedia.org/wiki/Transparency_(projection))! The transparent paper is marked with random dots and includes instructions on how to fold it up (your puzzle input). For example:

    6,10
    0,14
    9,10
    0,3
    10,4
    4,11
    6,0
    6,12
    4,1
    0,13
    10,12
    3,4
    3,0
    8,4
    1,10
    2,14
    8,10
    9,0
    
    fold along y=7
    fold along x=5
    

The first section is a list of dots on the transparent paper. `0,0` represents the top-left coordinate. The first value, `x`, increases to the right. The second value, `y`, increases downward. So, the coordinate `3,0` is to the right of `0,0`, and the coordinate `0,7` is below `0,0`. The coordinates in this example form the following pattern, where `#` is a dot on the paper and `.` is an empty, unmarked position:

    ...#..#..#.
    ....#......
    ...........
    #..........
    ...#....#.#
    ...........
    ...........
    ...........
    ...........
    ...........
    .#....#.##.
    ....#......
    ......#...#
    #..........
    #.#........
    

Then, there is a list of _fold instructions_. Each instruction indicates a line on the transparent paper and wants you to fold the paper _up_ (for horizontal `y=...` lines) or _left_ (for vertical `x=...` lines). In this example, the first fold instruction is `fold along y=7`, which designates the line formed by all of the positions where `y` is `7` (marked here with `-`):

    ...#..#..#.
    ....#......
    ...........
    #..........
    ...#....#.#
    ...........
    ...........
    -----------
    ...........
    ...........
    .#....#.##.
    ....#......
    ......#...#
    #..........
    #.#........
    

Because this is a horizontal line, fold the bottom half _up_. Some of the dots might end up overlapping after the fold is complete, but dots will never appear exactly on a fold line. The result of doing this fold looks like this:

    #.##..#..#.
    #...#......
    ......#...#
    #...#......
    .#.#..#.###
    ...........
    ...........
    

Now, only `17` dots are visible.

Notice, for example, the two dots in the bottom left corner before the transparent paper is folded; after the fold is complete, those dots appear in the top left corner (at `0,0` and `0,1`). Because the paper is transparent, the dot just below them in the result (at `0,3`) remains visible, as it can be seen through the transparent paper.

Also notice that some dots can end up _overlapping_; in this case, the dots merge together and become a single dot.

The second fold instruction is `fold along x=5`, which indicates this line:

    #.##.|#..#.
    #...#|.....
    .....|#...#
    #...#|.....
    .#.#.|#.###
    .....|.....
    .....|.....
    

Because this is a vertical line, fold _left_:

    #####
    #...#
    #...#
    #...#
    #####
    .....
    .....
    

The instructions made a square!

The transparent paper is pretty big, so for now, focus on just completing the first fold. After the first fold in the example above, _`17`_ dots are visible - dots that end up overlapping after the fold is completed count as a single dot.

_How many dots are visible after completing just the first fold instruction on your transparent paper?_

### Part Two

<span title="How can you fold it that many times? You tell me, I'm not the one folding it.">Finish folding</span> the transparent paper according to the instructions. The manual says the code is always _eight capital letters_.

_What code do you use to activate the infrared thermal imaging camera system?_


In [1]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [3]:
x = np.array([np.array([int(line.split(",")[0]),int(line.split(",")[1])],dtype=np.int64) for line in rawstring.splitlines() if len(line) != 0 and not ("fold" in line)],dtype=np.int64)

In [4]:
folds = [(line.split(" ")[-1].split("=")[0],int(line.split(" ")[-1].split("=")[1])) for line in rawstring.splitlines() if ("fold" in line)]
folds

[('x', 655),
 ('y', 447),
 ('x', 327),
 ('y', 223),
 ('x', 163),
 ('y', 111),
 ('x', 81),
 ('y', 55),
 ('x', 40),
 ('y', 27),
 ('y', 13),
 ('y', 6)]

In [5]:
max_x, max_y = np.max(x,axis=0)
arr = np.zeros(shape=(max_x+1,max_y+1),dtype=bool)
for i,j in x:
    arr[i,j] = True

part 1

In [6]:
foldpos = folds[0][1]
left_arr = arr[0:foldpos,:]
right_arr = np.flipud(arr[foldpos+1:,:])
(left_arr+right_arr).sum()

631

In [7]:
def perform_folds(arr,folds):
    inter_arr = copy.deepcopy(arr)
    print(inter_arr.shape)
    for dir, pos in folds:
        print(inter_arr.shape)
        if dir == 'x':
            left_arr  = inter_arr[0:pos,:]
            right_arr = np.flipud(inter_arr[pos+1:,:])
            inter_arr = left_arr + right_arr
        if dir == 'y':
            left_arr  = inter_arr[:,0:pos]
            right_arr = np.fliplr(inter_arr[:,pos+1:])
            inter_arr = left_arr + right_arr
    return inter_arr

res = perform_folds(arr, folds)

(1311, 895)
(1311, 895)
(655, 895)
(655, 447)
(327, 447)
(327, 223)
(163, 223)
(163, 111)
(81, 111)
(81, 55)
(40, 55)
(40, 27)
(40, 13)


In [8]:
letters = np.zeros_like(res, dtype=np.character)
letters[res == True] = '#'
letters[res == False] = ' '
def printL(x = letters[0:5,:].transpose()):
    for i in range(x.shape[0]):
        print("".join([str(c)[2] for c in x[i,:]]))
def printAll(j =letters):
    for i in range(int(j.shape[0]/5)):
        printL(letters[i*5:(i+1)*5,:].transpose())
        print()
printAll()

#### 
#    
###  
#    
#    
#### 

#### 
#    
###  
#    
#    
#    

#    
#    
#    
#    
#    
#### 

#### 
#    
###  
#    
#    
#    

  ## 
   # 
   # 
   # 
#  # 
 ##  

 ##  
#  # 
#    
# ## 
#  # 
 ### 

###  
#  # 
#  # 
###  
# #  
#  # 

#### 
#    
###  
#    
#    
#    

